In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import pandas
import sklearn
import tensorflow as tf
from tensorflow.contrib import learn
import pandas as pd
import nltk
import re
from sklearn import cross_validation

In [2]:
def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [13]:
#making data frame

f=open("/home/prayalankar/data/train_1000.label","r+b")
col=['category','label','data']
df=pd.DataFrame(columns=col)
df

while True:
    as1=f.readline()
    if as1=='':
        break
    as1=nltk.word_tokenize(as1)
    str1=' '.join(as1[3:])
    df.loc[len(df)]=[as1[0],as1[2],str1]

In [4]:
#separating the text data

x=df.data
de=[]
for p in x:
    de.append(p)

In [5]:
#final cleaned text
x_text=[clean_str(p) for p in de]

In [6]:
#categorising the labels to int 
number=sklearn.preprocessing.LabelEncoder()
df['col3']=number.fit_transform(df['category'])
ye=df.col3

In [7]:
DOC_LEN = 10
E_SIZE = 50
n_words = 0

In [8]:
# a method which creates the word vectors and then used GRU cell to store it . This is later passed to supervised algo for training

def rnn_logis(x, y):
    word_vectors = learn.ops.categorical_variable(x, n_classes=n_words,
                                                  embedding_size=E_SIZE, name='words')
    word_list = tf.unpack(word_vectors, axis=1)
    cell = tf.nn.rnn_cell.GRUCell(E_SIZE)
    _, encoding = tf.nn.rnn(cell, word_list, dtype=tf.float32)
    target = tf.one_hot(y, 6, 1, 0)
    prediction, loss = learn.models.logistic_regression(encoding, target)
    train_op = tf.contrib.layers.optimize_loss(
        loss, tf.contrib.framework.get_global_step(),
        optimizer='Adam', learning_rate=0.01)
    
    return {'class': tf.argmax(prediction, 1), 'prob': prediction}, loss, train_op

In [9]:
#converting text features into numerical values
global n_words
x_train = pandas.DataFrame(x_text[100:])[0]
y_train = pandas.Series(ye[100:])
x_test = pandas.DataFrame(x_text[:100])[0]
y_test = pandas.Series(ye[:100])
#using tensorflow Vocabulary for embedding text data
vocab_processor = learn.preprocessing.VocabularyProcessor(DOC_LEN)
x_train = np.array(list(vocab_processor.fit_transform(x_train)))
x_test = np.array(list(vocab_processor.transform(x_test)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 2588


In [10]:
classifier = learn.Estimator(model_fn=rnn_logis)
classifier.fit(  x_train, y_train, steps=100)
y_predicted = [

    p['class'] for p in classifier.predict(x_test, as_iterable=True)]

In [12]:
score = sklearn.metrics.accuracy_score(y_test, y_predicted)
score

0.76000000000000001